# 5_Regression&Autoregression

以上我們有了data,並組成data loader，我們使用剛剛loader可以來訓練一些模型。

這邊我們用些簡單的迴歸模型來試試看:
- Linear Regression
- Autoregression

**先import一些套件**

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
from plotly import express as px

import numpy as np
import tensorflow.data as tfd

In [ ]:
# Colab使用要更換statsmodels版本 ，安裝完請重新啟動執行階段
!pip uninstall -y statsmodels
!pip install statsmodels==0.11.1

**給一些必要function: 畫圖、合成資料產生、window data loader產生、評估function**

In [ ]:
def plot_series(time, series, start=0, end=None, labels=None, title=None):
    #  Visualizes time series data
    # Args:
    #  time (array of int) - 時間點, 長度為T
    #  series (list of array of int) - 時間點對應的資料列表，列表內時間序列數量為D，
    #                                  每筆資料長度為T，若非為列表則轉為列表
    #  start (int) - 開始的資料序(第幾筆)
    #  end (int) -   結束繪製的資料序(第幾筆)
    #  labels (list of strings)- 對於多時間序列或多維度的標註
    #  title (string)- 圖片標題

    # 若資料只有一筆，則轉為list
    if type(series) != list:
        series = [series]

    if not end:
        end = len(series[0])

    if labels:
        # 設立dictionary, 讓plotly畫訊號線時可以標註label
        dictionary = {"time": time}
        for idx, l in enumerate(labels):
            # 截斷資料，保留想看的部分，並分段紀錄於dictionary中
            dictionary.update({l: series[idx][start:end]})
        # 畫訊號線
        fig = px.line(dictionary,
                      x="time",
                      y=list(dictionary.keys())[1:],
                      width=1000,
                      height=400,
                      title=title)
    else:
        # 畫訊號線
        fig = px.line(x=time, y=series, width=1000, height=400, title=title)
    fig.show()


# 合成資料生成
def trend(time, slope=0):
    # 產生合成水平直線資料，其長度與時間等長，直線趨勢與設定slope相同
    # Args:
    #  time (array of int) - 時間點, 長度為T
    #  slope (float) - 設定資料的傾斜程度與正負
    # Returns:
    #  series (array of float) -  產出slope 與設定相同的一條線

    series = slope * time

    return series


def seasonal_pattern(season_time, pattern_type='triangle'):
    # 產生某個特定pattern，
    # Args:
    #  season_time (array of float) - 周期內的時間點, 長度為T
    #  pattern_type (str) -  這邊提供triangle與cosine
    # Returns:
    #  data_pattern (array of float) -  根據自訂函式產出特定的pattern

    # 用特定function生成pattern
    if pattern_type == 'triangle':
        data_pattern = np.where(season_time < 0.5,
                                season_time*2,
                                2-season_time*2)
    if pattern_type == 'cosine':
        data_pattern = np.cos(season_time*np.pi*2)

    return data_pattern


def seasonality(time, period, amplitude=1, phase=30, pattern_type='triangle'):
    # Repeats the same pattern at each period
    # Args:
    #   time (array of int) - 時間點, 長度為T
    #   period (int) - 週期長度，必小於T
    #   amplitude (float) - 序列幅度大小
    #   phase (int) - 相位，為遞移量，正的向左(提前)、負的向右(延後)
    #   pattern_type (str) -  這邊提供triangle與cosine
    # Returns:
    #   data_pattern (array of float) - 有指定周期、振幅、相位、pattern後的time series

    # 將時間依週期重置為0
    season_time = ((time + phase) % period) / period

    # 產生週期性訊號並乘上幅度
    data_pattern = amplitude * seasonal_pattern(season_time, pattern_type)

    return data_pattern


def noise(time, noise_level=1, seed=None):
    # 合成雜訊，這邊用高斯雜訊，機率密度為常態分布
    # Args:
    #   time (array of int) - 時間點, 長度為T
    #   noise_level (float) - 雜訊大小
    #   seed (int) - 同樣的seed可以重現同樣的雜訊
    # Returns:
    #   noise (array of float) - 雜訊時間序列

    # 做一個基於某個seed的雜訊生成器
    rnd = np.random.RandomState(seed)

    # 生與time同長度的雜訊，並且乘上雜訊大小 (不乘的話，標準差是1)
    noise = rnd.randn(len(time)) * noise_level

    return noise


def toy_generation(time=np.arange(4 * 365),
                   bias=500.,
                   slope=0.1,
                   period=180,
                   amplitude=40.,
                   phase=30,
                   pattern_type='triangle',
                   noise_level=5.,
                   seed=2022):
    signal_series = bias\
                  + trend(time, slope)\
                  + seasonality(time,
                                period,
                                amplitude,
                                phase,
                                pattern_type)
    noise_series = noise(time, noise_level, seed)

    series = signal_series+noise_series
    return series


# Dataset
def win_ar_ds(series, size, shift=1):
    # 輸出Window-wise Forcasting Dataset
    # Args:
    #   series (array of float) - 時序資料, 長度為T
    #   size (int) - Window大小
    #   shift (int) - 每個window起始點間距
    # Returns:
    #   (tf.data.Dataset(母類名稱，切確type為MapDataset)) -
    #   - 一個一次生一個window的生成器

    ds = tfd.Dataset.from_tensor_slices(series)
    ds = ds.window(size=size+1, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda ds: ds.batch(size+1))
    return ds.map(lambda x: (x[:-1], x[-1:]))


def regressor_ds(*regressors, series):
    # 輸出Window-wise Regressor Forcasting Dataset
    # Args:
    #   regressors (arguments of array of float) - 多個迴歸因子，每個長度為T
    #   series (array of float) - 預測對象，長度
    # Returns:
    #   (tf.data.Dataset(母類名稱，切確type為TensorSliceDataset)) -
    #   - 一次生regressors和time series的dataset

    ds = tfd.Dataset.from_tensor_slices((np.stack(regressors, -1), series))
    return ds


# 評估function
def MAE(pred, gt):
    # 計算Mean Absolute Error
    # Args:
    #  pred (array of float) - 預測資料
    #  gt (array of float) - 答案資料
    # Returns:
    #  計算結果 (float)
    return abs(pred-gt).mean()


def MSE(pred, gt):
    # 計算Mean Square Error
    # Args:
    #  pred (array of float) - 預測資料
    #  gt (array of float) - 答案資料
    # Returns:
    #  計算結果 (float)
    return pow(pred-gt, 2).mean()


def R2(pred, gt):
    # 計算R square score
    # Args:
    #  pred (array of float) - 預測資料
    #  gt (array of float) - 答案資料
    # Returns:
    #  計算結果 (float)
    return 1-pow(pred-gt, 2).sum()/pow(gt-gt.mean(), 2).sum()

In [ ]:
def split(x, train_size):
    return x[..., :train_size], x[..., train_size:]


# 先合成資料，還有作資料分割
time = np.arange(4*365)  # 定義時間點
series_sample = toy_generation(time, pattern_type='cosine')  # 這就是我們合成出來的資料

time_train, time_test = split(time, 365*3)
series_train, series_test = split(series_sample, 365*3)

# 另外也加上輔助資料
cos_train = seasonality(time_train, 180, 1., 30, 'cosine')
cos_test = seasonality(time_test, 180, 1., 30, 'cosine')

In [ ]:
import tensorflow as tf
from tensorflow.keras import models, layers, losses, optimizers

In [ ]:
# 用各種regressor predict資料的training set
train_ds_r = regressor_ds(time_train,
                          cos_train,
                          series=series_train)  # 切time series
train_loader_r = train_ds_r.cache()\
    .shuffle(1000).batch(32, drop_remainder=True).prefetch(-1)

# 用各種regressor predict資料的testing set
test_ds_r = regressor_ds(time_test,
                         cos_test,
                         series=series_test)  # 切time series
test_loader_r = test_ds_r.batch(32).prefetch(-1)

In [ ]:
for x, y in train_loader_r:
    pass
print(x.shape, y.shape)

In [ ]:
cos_train = seasonality(time_train, 180, 1., 45, 'cosine')
plt.figure(figsize=(20, 5))
plt.plot(cos_train, 'b', linewidth=3)

## Linear Regression

前面ML的課程已經教過Linear Regression的課程，

其中多變量迴歸時是透過轉置矩陣w與偏移b可以預測下個時間點的值

$y=w^Tx+b$

透過訓練參數w與b可以使系統更加能擬合feature與預測值之間關係。

<img src=https://i.imgur.com/0B4K969.png width=200 align=left>




而Linear time regression中我們使用$t_{n-W:n}$代入input x的部分，計算output $y_n$應該要是多少

<img src=https://i.imgur.com/jqHvsVw.png width=500 align=left>

而在示範的training data中除了time regression我們還加上cosine波形作為另一個regression feature (又稱regressor)。

我們的t是前面```time_train```的部分，cosine波形是前面```cos_train```，y是前面```series_train```的部分

### Build TF2 Model

而我們這邊可以用一個一層且無activation的Dense層來完成這個功能，而且我們前面有幾個layer

In [ ]:
model = models.Sequential([
    layers.Flatten(input_shape=[2], data_format="channels_first"),
    layers.Dense(1, activation=None)
])

In [ ]:
model.summary()
# 共有window_size*K+1個parameter 就看要放幾個regressor,最後一個parameter是bias

In [ ]:
opt = optimizers.Adam(learning_rate=1e-1)
model.compile(loss="mse", optimizer=opt)

### Training

In [ ]:
history = model.fit(
    train_loader_r,
    epochs=400,
    verbose=2,
    callbacks=[
        tf.keras.callbacks.ReduceLROnPlateau(monitor='loss'),
        tf.keras.callbacks.EarlyStopping(
            monitor='loss',
            patience=20,
            verbose=2)])

可以看一下model的weight

In [ ]:
# 第一個是time trend的weight，第二個是cosine wave 的trend
model.weights[0].numpy()

這邊故意調整weight的順序，使得前面一半是linear trend相關的weight，後面一半是cosine相關的weight。

可以看出這兩者的量值跟我們預設的很接近

In [ ]:
# bias
model.weights[1].numpy()

bias則是很逼近我們預設的level

### Evaluate

In [ ]:
model.evaluate(test_loader_r)

基本上就是很圓滑的線，因為我們只用了linear trend以及cosine而已

結果就還好

In [ ]:
forcast = model.predict(test_loader_r)[:, 0]
time_for_view = time_test

plot_series(time_for_view,
            [forcast, series_test],
            labels=['prediction', 'ground truth'])

這邊我們把那些訓練好的weight套上個別regressor，並扣除掉訓練好的bias的影響

畫出來看看:

$trend[t]=time[t]*weight_{0}$

$seasonality[t]=cos[t]*weight_{1}$

$y'[t]=y[t]-bias$

In [ ]:
plot_series(time_for_view,
            [time_test*model.weights[0][0].numpy(),
             cos_test*model.weights[0][1].numpy(),
             series_test-model.weights[1][0].numpy()],
            labels=['weighted time', 'weighted cosine', 'ground truth'])

In [ ]:
# 算出R2 score來看看
R2(forcast, series_test)

### Exercise
請試試看不同regressor對regression的影響，可以注意到對weight跟bias的影響

e.g. 將```pattern_type```換成```triangle```

## Autoregressive Model

Autoregression假設現在的序列與前面有限個時間點的數個序列有關：

$y_t=w_0 y_{t-1}+w_1 y_{t-2}+...+ w_T y_{t-T} $

其實就是對前面的時間點套用一個固定的加權和，而這個加權的權重是可以訓練的。

也可以看作是套用某個波形作1D convolution來預測未來序列，這樣在有一定周期性時是有幫助的。

In [ ]:
# 這邊要用不一樣的dataset
window_size = 7

# 用資料predict資料的training set
train_ds = win_ar_ds(series_train, size=window_size)  # 切time series
train_loader = train_ds.cache()\
    .shuffle(1000).batch(32, drop_remainder=True).prefetch(-1)

# 用資料predict資料的testing set
test_ds = win_ar_ds(series_test, size=window_size)  # 切time series
test_loader = test_ds.batch(32).prefetch(-1)
for x, y in train_loader:
    pass
print(x.shape, y.shape)

### Build TF2 Model

而我們這邊可以用一個一層且無activation的Dense層來完成一個AR model

In [ ]:
model_ar = models.Sequential([
    layers.Dense(1, input_shape=[window_size], activation=None)
])

In [ ]:
model_ar.summary()
# 共有window_size+1個parameter,最後一個parameter是bias

In [ ]:
opt = optimizers.Adam(learning_rate=1e-2)
model_ar.compile(loss="mse", optimizer=opt)

### Training

In [ ]:
history = model_ar.fit(
    train_loader,
    epochs=400,
    verbose=2,
    callbacks=[
        tf.keras.callbacks.ReduceLROnPlateau(monitor='loss'),
        tf.keras.callbacks.EarlyStopping(
            monitor='loss',
            patience=20,
            verbose=2)])

可以看一下model的weight

In [ ]:
# weight
plt.bar(np.arange(window_size), model_ar.weights[0].numpy().squeeze())

它的weight與AutoCorrelated Function正相關，也就是前面講的convolution時用到的波形。

In [ ]:
# bias
model_ar.weights[1]

### Evaluate

In [ ]:
model_ar.evaluate(test_loader)

In [ ]:
forcast = model_ar.predict(test_loader)[:, 0]
ground_truth_for_view = series_test[window_size:]
time_for_view = time_test[window_size:]

plot_series(time_for_view,
            [forcast, ground_truth_for_view],
            labels=['prediction', 'ground truth'])

這邊我們使用autoregression的結果，因為noise無法擬和，所以會受到干擾

而且跟SES很像，如果預估的時間很長，則預估越來越不准

In [ ]:
R2(forcast, ground_truth_for_view)

## Autoregressive Data

當資料有短期的autoregressive效應時，目前資料會受過往資料影響較多。

我們拿一個autoregressive kernel與一組起始的數值來試試看

In [ ]:
kernal = np.array([-0.5, 0.4, -0.3, 0.4, 0.5])
kernal = kernal/np.linalg.norm(kernal)

series = [2, 2, 2, 2, 2]
# series = [1, 2, 3, 4, 5]
# series = [5, 4, 3, 2, 1]
for i in range(80):
    last = np.array(series[-5:])
    series.append(kernal@last)

In [ ]:
plot_series(np.arange(len(series)), np.array(series))

其實可以看出來，autoregression結果在哪種起始狀態都無所謂，只要不是全為0，只要weight一樣最後都會converge到一樣的pattern

## Autoregressive Integrated Moving Average (ARIMA)

這邊AR model 可延伸出ARMA, ARIMA或者更後面的SARIMA model。

ARMA就是分成兩個AR model對資料做擬合：
1. 第一部分是做time series的AR model；
2. 另一部分是residual 的AR model

$F_𝑛=𝛽_1 𝑦_{𝑛−1}+𝛽_2 𝑦_{𝑛−2}+…+𝛽_𝑝 𝑦_{𝑛−𝑝}\ (part 1)$

  $+𝜖_𝑛+𝜃_1 𝜖_{𝑛−1}+𝜃_2 𝜖_{𝑛−2}+…+𝜃_𝑞 𝜖_{𝑛−𝑞}\ (part 2)$
  
這residual term 是從原series扣掉AR model forecast的結果產生的序列

(較為簡單的版本是用moving average取代AR model，後來比較常用AR model做第一階段forecast)

$𝜖_𝑛=𝑦_𝑛− 𝛽_1 𝑦_{𝑛−1}+𝛽_2 𝑦_{𝑛−2}+…+𝛽_𝑝 𝑦_{𝑛−𝑝} $


<img src=https://i.imgur.com/x4oHilw.png width=400 align=left>

那整體模型則是由原series經過AR模型得到AR forecast，並與原series相減得到residual
residual 再經由MA模型(part2)得到MA forecast
最後這兩個forecast加在一起就是ARMA model的預測

至於超參數p與q則是需要一些對data的了解或嘗試取得最佳解。

<img src=https://i.imgur.com/HCtJarZ.png width=400 align=left>

ARIMA就是先做完differencing再做前面的ARMA，Differencing之後可以去掉trend，所以要求只需要保持不要有seasonality就好。

In [ ]:
# 用Sequencial方式也組成MA model
# 這邊參數數量q使用與AR model一樣
model_ma = models.Sequential([
    layers.Dense(1, input_shape=[window_size], activation=None, use_bias=False)
])

In [ ]:
model_ma.summary()
# 共有window_size個parameter

In [ ]:
opt = optimizers.Adam(learning_rate=1e-2)
model_ma.compile(loss="mse", optimizer=opt)

In [ ]:
# 這邊要重做一個dataset

# 取出residual
residual_train = series_train[window_size:]\
    - model_ar.predict(train_ds.batch(32)).squeeze()
residual_test = series_test[window_size:]\
    - model_ar.predict(test_ds.batch(32)).squeeze()

# 組成dataloader
residual_train_loader = win_ar_ds(residual_test, size=window_size)\
    .cache().shuffle(1000).batch(32, drop_remainder=True).prefetch(-1)

residual_test_loader = win_ar_ds(residual_test, size=window_size)\
    .batch(32).prefetch(-1)

In [ ]:
history = model_ma.fit(
    residual_train_loader,
    epochs=400,
    verbose=2,
    callbacks=[
        tf.keras.callbacks.ReduceLROnPlateau(monitor='loss'),
        tf.keras.callbacks.EarlyStopping(
            monitor='loss',
            patience=20,
            verbose=2)])

### Evaluate

In [ ]:
model_ma.evaluate(residual_test_loader)

In [ ]:
forcast_ar = model_ar.predict(test_loader)[window_size:, 0]  # AR 預測
forcast_ma = model_ma.predict(residual_test_loader)[:, 0]  # MA 預測
forcast = forcast_ar+forcast_ma  # 加起來

ground_truth_for_view = series_test[window_size*2:]

time_for_view = time_test[window_size*2:]

plot_series(time_for_view,
            [forcast, ground_truth_for_view],
            labels=['prediction', 'ground truth'])

In [ ]:
# 與純粹AR 模型相比 可以得到較好一點點的結果
R2(forcast, ground_truth_for_view)

ARMA與ARIMA模型只差differencing而已，可以使用一些現成的套件完成

## Call Function from Module

statsmodels有提供ARIMA的model，對training的時間點資料進行擬合，訓練出differeced data的AR及MA模型。

In [ ]:
from statsmodels.tsa.arima_model import ARIMA

In [ ]:
# 組成ARIMA模型
# Args:
#  endog (array of float) - 時間序列
#  order (tuple of (int, int, int) ) - ARIMA order, 包含p,d,q
#    p - AR model的coefficient數量
#    d - differencing次數
#    q - 擬合residual term的coefficient數量
# Returns:
#  ARIMA模型物件 (statsmodels.tsa.arima_model)

# endog: Input data
arima = ARIMA(endog=series_train, order=(7, 1, 7))

用p, d, q 參數組和，除ARIMA model外也可以組成AR Model和ARMA model:
- AR: (p,0,0) 沒有residual也沒有differencing
- ARMA: (p,0,q) 有residual沒有differencing
- ARIMA: (p,d,q) 有residual也有differencing

### Training

In [ ]:
mdl = arima.fit()
print(mdl.summary())

Summary報告中可呈現各參數的數值
- const: bias項
- ar.L*.D.y:
    - coef: AR model的參數值
    - z: regression 參數的 z 統計值
- ma.L*.D.y:
    - coef: MA model的參數值
    - z: regression 參數的 z 統計值

In [ ]:
forcast = mdl.forecast(len(series_test))[0]

plot_series(time_test,
            [forcast, series_test],
            labels=['prediction', 'ground truth'])

In [ ]:
R2(forcast, series_test)

### References
* statsmodes官網: https://www.statsmodels.org/dev/generated/statsmodels.tsa.arima.model.ARIMA.html
* https://medium.com/analytics-vidhya/arima-model-from-scratch-in-python-489e961603ce
* https://www.nbshare.io/notebook/136553745/Time-Series-Analysis-Using-ARIMA-From-StatsModels/